# Multi Modal Analysis Integrated Module

In [4]:
import tkinter as tk
from tkinter import simpledialog
import numpy as np
import torch
import torch.nn as nn
import tensorflow as tf
import cv2
import joblib
from sklearn.preprocessing import StandardScaler
import warnings

warnings.filterwarnings("ignore")  # Suppress all warnings


# Define the LSTM model architecture
class EmotionLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes, dropout=0.5):
        super(EmotionLSTM, self).__init__()
        self.lstm = nn.LSTM(
            input_size, hidden_size, num_layers,
            batch_first=True, bidirectional=True,
            dropout=dropout if num_layers > 1 else 0
        )
        self.attention = nn.Sequential(
            nn.Linear(hidden_size * 2, 1),
            nn.Tanh()
        )
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(hidden_size * 2, num_classes)

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        attention_weights = self.attention(lstm_out)
        attention_weights = torch.softmax(attention_weights, dim=1)
        context = torch.sum(attention_weights * lstm_out, dim=1)
        out = self.dropout(context)
        out = self.fc(out)
        return out


# Load models
face_emotion_model = tf.keras.models.load_model(r'C:/Users/reeva/Desktop/690/best_model.keras')
sleep_model = joblib.load(r'C:/Users/reeva/Desktop/690/sleep-edf-database-expanded-1.0.0-20241127T213628Z-001/sleep-edf-database-expanded-1.0.0/multi_label_model.joblib')

# Load LSTM model with matching architecture
input_size = 2
hidden_size = 64
num_layers = 2
num_classes = 4
lstm_model = EmotionLSTM(input_size, hidden_size, num_layers, num_classes)
checkpoint = torch.load(r'C:/Users/reeva/Desktop/690/best_emotion_model.pth', map_location=torch.device('cpu'))
lstm_model.load_state_dict(checkpoint['model_state_dict'])
lstm_model.eval()

# Create and fit a scaler with simulated data
scaler = StandardScaler()
scaler.fit([[0.1, 0.2, 0.3, 0.4, 0.5, 0.6], [0.5, 0.4, 0.3, 0.2, 0.1, 0.0]])


# Emotion mappings
emotion_mappings = {
    "face": {'Happy': 'Happy', 'Sad': 'Stressed', 'Neutral': 'Bored', 'Angry': 'Stressed', 'Surprise': 'Happy'},
    "lstm": {'Baseline': 'Bored', 'Stress': 'Stressed', 'Amusement': 'Happy', 'Meditation': 'Relaxed'},
    "sleep": {'happiness': 'Happy', 'sadness': 'Sad', 'anger': 'Angry', 'surprise': 'Happy', 'fear': 'Stressed'}
}

common_emotions = ['Happy', 'Sad', 'Neutral', 'Angry', 'Relaxed', 'Stressed', 'Bored']


# Map outputs to common emotions
def map_emotion_values(module_values, mapping):
    mapped_values = {emotion: 0.0 for emotion in common_emotions}
    total_value = 0.0  # Track total for normalization

    for emotion, value in module_values.items():
        if emotion in mapping:
            common_emotion = mapping[emotion]
            scalar_value = float(value) if not isinstance(value, (list, np.ndarray)) else float(np.mean(value))
            mapped_values[common_emotion] += scalar_value
            total_value += scalar_value

    if total_value > 0:
        for emotion in mapped_values:
            mapped_values[emotion] /= total_value

    return mapped_values


# Combine probabilities with weights
def combine_emotion_values(mapped_values_list, weights):
    combined_values = {emotion: 0.0 for emotion in common_emotions}
    for mapped_values, weight in zip(mapped_values_list, weights):
        for emotion, value in mapped_values.items():
            combined_values[emotion] += value * weight
    return combined_values


# Face emotion detection
def detect_face_emotion():
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Webcam not accessible.")
        return {'Neutral': 1.0}

    print('Initializing webcam...')
    print("Press 'c' to capture a photo.")
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("Frame capture failed.")
            break

        cv2.imshow('Press "c" to capture', frame)
        if cv2.waitKey(1) & 0xFF == ord('c'):
            cap.release()
            cv2.destroyAllWindows()
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            roi_gray = cv2.resize(gray, (48, 48), interpolation=cv2.INTER_AREA)
            roi_rgb = np.expand_dims(np.stack([roi_gray] * 3, axis=-1) / 255.0, axis=0)
            predictions = face_emotion_model.predict(roi_rgb)[0]
            face_results = dict(zip(['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral'], predictions))
            #print("Face Detection Results:", face_results)  # Debug print
            return face_results

    cap.release()
    cv2.destroyAllWindows()
    return {'Neutral': 1.0}


# LSTM emotion detection
def detect_lstm_emotion(input_data):
    tensor_data = torch.tensor(input_data, dtype=torch.float32).unsqueeze(0)
    output = lstm_model(tensor_data)
    probabilities = torch.softmax(output, dim=1).detach().numpy()[0]
    lstm_results = dict(zip(['Baseline', 'Stress', 'Amusement', 'Meditation'], probabilities))
    #print("LSTM Detection Results:", lstm_results)  # Debug print
    return lstm_results


# Sleep emotion detection
def detect_sleep_emotion(input_data):
    input_array = np.array(input_data).reshape(1, -1)
    input_normalized = scaler.transform(input_array)
    predictions = sleep_model.predict_proba(input_normalized)[0]
    sleep_results = dict(zip(['happiness', 'sadness', 'anger', 'surprise', 'fear'], predictions))
    #print("Sleep Detection Results:", sleep_results)  # Debug print
    return sleep_results


# Display results
def display_results(face_results, lstm_results, sleep_results, combined, final_emotion):
    #print("\n--- Debugging: Raw Inputs to display_results ---")
    #print("Face Results:", face_results)
    #print("LSTM Results:", lstm_results)
    #print("Sleep Results:", sleep_results)
    #print("Combined Results:", combined)
    #print("Final Emotion:", final_emotion)

    print("\n--- Emotion Detection Results ---")
    print("Face Emotion Results:", face_results)
    print("LSTM Emotion Results:", lstm_results)
    print("Sleep Emotion Results:", sleep_results)
    print("Combined Results:", combined)
    print(f"Final Emotion: {final_emotion}")


# Main function
def run_emotion_detection():
    root = tk.Tk()
    root.withdraw()

    # Face emotion detection
    face_emotion = detect_face_emotion()
    if not face_emotion:
        print("Error: Face emotion detection failed!")

    # LSTM emotion detection
    lstm_input = simpledialog.askstring(
        "Input",
        "Provide physiological signals:\n"
        "1. ECG: Electrical activity of the heart.\n"
        "2. Resp: Breathing rate.\n"
        "Input Format: <ECG_signal>, <Resp_signal>\n"
        "Example: 0.85, 0.65"
    )
    lstm_emotion = detect_lstm_emotion([list(map(float, lstm_input.split(',')))])
    if not lstm_emotion:
        print("Error: LSTM emotion detection failed!")

    # Sleep emotion detection
    sleep_input = simpledialog.askstring(
        "Input",
        "Provide six sleep metrics:\n"
        "1. prop_W: Wake.\n"
        "2. prop_1: NREM Stage 1.\n"
        "3. prop_2: NREM Stage 2.\n"
        "4. prop_3: NREM Stage 3.\n"
        "5. prop_4: NREM Stage 4.\n"
        "6. prop_R: REM Sleep.\n"
        "Input Format: <prop_W>, <prop_1>, <prop_2>, <prop_3>, <prop_4>, <prop_R>\n"
        "Example: 0.2, 0.1, 0.4, 0.2, 0.1, 0.5"
    )
    sleep_emotion = detect_sleep_emotion(list(map(float, sleep_input.split(','))))
    if not sleep_emotion:
        print("Error: Sleep emotion detection failed!")

    # Map values
    mapped_face = map_emotion_values(face_emotion, emotion_mappings['face'])
    mapped_lstm = map_emotion_values(lstm_emotion, emotion_mappings['lstm'])
    mapped_sleep = map_emotion_values(sleep_emotion, emotion_mappings['sleep'])

    # Combine and determine final emotion
    weights = [0.4, 0.3, 0.3]
    combined = combine_emotion_values([mapped_face, mapped_lstm, mapped_sleep], weights)
    final_emotion = max(combined, key=combined.get)

    # Display results
    display_results(mapped_face, mapped_lstm, mapped_sleep, combined, final_emotion)

    return final_emotion  # Returning the final emotion


In [ ]:
# Call the emotion detection function
detected_emotion = run_emotion_detection()
print(f"\nDetected Emotion: {detected_emotion}")